# TODO

- Clean up code
  - Function for plotting
  - Better setup for indexing
- Add math 

$$
\begin{bmatrix}
x \\
y \\
\end{bmatrix}
$$

$$
\begin{bmatrix}
x \\
0 \\
0 \\
y \\
\end{bmatrix}
$$

$$
\begin{bmatrix}
I   & 0   & 0   & 0 \\
f_1 & 0   & 0   & 0 \\
0   & f_2 & 0   & 0 \\
0   & 0   & f_3 & 0 \\
\end{bmatrix}
$$


# Load data

In [1]:
import torch
import pandas as pd

from iterativennsimple.Sequential2D import Sequential2D, Identity
from iterativennsimple.Sequential1D import Sequential1D

from iterativennsimple.SparseLinear import SparseLinear

from iterativennsimple.utils.plot_2D_start_end import plot_2D_start_end

In [2]:
# Turn a pandas dataframe into a pytorch tensor
def df_to_tensor(df):
    return torch.tensor(df.values, dtype=torch.float32)

In [3]:
#name = 'regression_line'
#name = 'pca_line'
#name = 'circle'
#name = 'regression_circle'
#one_hot_y = False

name = 'MNIST'
one_hot_y = True

# Read the start data
z_start = pd.read_parquet(f'../data/processed/{name}_start.parquet')
# Read the target data
z_target = pd.read_parquet(f'../data/processed/{name}_target.parquet')

## Data preprocessing

In [4]:
z_start_tensor = df_to_tensor(z_start)
z_target_tensor = df_to_tensor(z_target)

# Only use the given number of sample
max_num_samples = 1000
num_samples = min(max_num_samples, z_start_tensor.shape[0])
z_start_tensor = z_start_tensor[:num_samples]
z_target_tensor = z_target_tensor[:num_samples]

# compute a mask for the data where every entry in each column is the same
# this is used to remove columns that are constant
mask = (z_start_tensor == z_target_tensor).all(axis=0)
mask.shape
x_mask = mask
y_mask = ~mask

# print out the shape of the masks
print(x_mask.shape, y_mask.shape)

# print out number of true entries in each mask
print(x_mask.sum(), y_mask.sum())


torch.Size([785]) torch.Size([785])
tensor(784) tensor(1)


# MLP model

In [5]:
input_size = int(x_mask.sum())
h1_size = 20
h2_size = 20
output_size = int(y_mask.sum())

x_idx = torch.arange(0, input_size)
h_idx = torch.arange(input_size, input_size+h1_size+h2_size)
y_idx = torch.arange(input_size+h1_size+h2_size, input_size+h1_size+h2_size+output_size)

total_size = input_size + h1_size + h2_size + output_size

iterations = 3

I = Identity(in_features=input_size, out_features=input_size)

l1 = SparseLinear.from_singleBlock(col_size=input_size, row_size=h1_size,  
                                   block_type='R=1.0', initialization_type='G=0.0,0.001', 
                                   optimized_implementation=True)
# l1 = torch.nn.Linear(in_features=input_size, out_features=h1_size)
f1 = Sequential1D(l1, 
                  torch.nn.ReLU(), 
                  in_features=input_size, out_features=h1_size)


l2 = SparseLinear.from_singleBlock(col_size=h1_size, row_size=h2_size,  
                                   block_type='R=1.0', initialization_type='G=0.0,0.001', 
                                   optimized_implementation=True)
# l2 = torch.nn.Linear(in_features=h1_size, out_features=h2_size)
f2 = Sequential1D(l2, 
                  torch.nn.ReLU(), 
                  in_features=h1_size,    out_features=h2_size)

f3 = SparseLinear.from_singleBlock(col_size=h2_size, row_size=output_size,  
                                   block_type='R=1.0', initialization_type='G=0.0,0.001', 
                                   optimized_implementation=True)
#f3 = torch.nn.Linear(in_features=h2_size,    out_features=output_size)    

in_features_list  = [input_size, h1_size, h2_size, output_size]
out_features_list = [input_size, h1_size, h2_size, output_size]
blocks = [[I,    None, None, None],
          [f1,   None, None, None],
          [None, f2,   None, None],
          [None, None, f3,   None]]

def transpose_blocks(blocks):
    return [[blocks[j][i] for j in range(len(blocks))] for i in range(len(blocks[0]))]

map = Sequential2D(
    in_features_list=in_features_list,
    out_features_list=out_features_list,
    blocks=transpose_blocks(blocks)
)   

/home/rcpaffenroth/projects/iterativennsimple/.venv/lib/python3.10/site-packages/iterativennsimple/SparseLinear.py:102: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.sparse_trainable_values = torch.nn.parameter.Parameter(torch.tensor(sparse_trainable.values(), **factory_kwargs),


In [6]:
zh_start_tensor = torch.cat((z_start_tensor[:, x_mask],
                             torch.zeros(z_start_tensor.shape[0], len(h_idx)), 
                             z_start_tensor[:, y_mask]), dim=1)
zh_target_tensor = torch.cat((z_target_tensor[:, x_mask], 
                             torch.zeros(z_target_tensor.shape[0], len(h_idx)), 
                             z_target_tensor[:, y_mask]), dim=1)

In [7]:
# a dataloader which returns a batch of start and target data
class Data(torch.utils.data.Dataset):
    def __init__(self, z_start, z_target):
        self.z_start = z_start
        self.z_target = z_target
    def __len__(self):
        return len(self.z_start)
    def __getitem__(self, idx):
        return self.z_start[idx], self.z_target[idx]
    
train_data = Data(zh_start_tensor, zh_target_tensor)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=256, shuffle=True)

In [8]:
# Define the loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(map.parameters(), lr=0.001)

In [9]:
max_epochs = 100
# Train the model
for epoch in range(max_epochs):
    for batch_idx, (start, target) in enumerate(train_loader):
        optimizer.zero_grad()

        mapped = start

        loss = 0.0
        for i in range(iterations):
            mapped = map(mapped)

        loss = criterion(mapped[:, y_idx], target[:, y_idx])
        loss.backward()

        optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Batch {batch_idx}, Loss {loss.item()}')

Epoch 0, Batch 3, Loss nan
Epoch 10, Batch 3, Loss nan
Epoch 20, Batch 3, Loss nan
Epoch 30, Batch 3, Loss nan
Epoch 40, Batch 3, Loss nan
Epoch 50, Batch 3, Loss nan
Epoch 60, Batch 3, Loss nan
Epoch 70, Batch 3, Loss nan
Epoch 80, Batch 3, Loss nan
Epoch 90, Batch 3, Loss nan
